**Create DB on AWS**

In [1]:
! pip install psycopg2-binary pandas --quiet
import psycopg2
import pandas as pd
from psycopg2 import sql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 51.5 MB/s eta 0:00:00


In [ ]:
conn = psycopg2.connect(
    host="mlprojectdb.cx4k04y48cjc.eu-north-1.rds.amazonaws.com",
    dbname="postgres",
    user="postgres",
    password="mlpipeline",
    port=5432
)

conn.autocommit = True

cur = conn.cursor()
cur.execute("CREATE DATABASE telco_churn;")

cur.close()
conn.close()


**Create Table on DB**

In [ ]:
import psycopg2

# Hardcoded credentials (as requested)
HOST = "mlprojectdb.cx4k04y48cjc.eu-north-1.rds.amazonaws.com"
PORT = 5432
USER = "postgres"
PASSWORD = "mlpipeline"
MASTER_DB = "postgres"   # connect here to create the database
DB = "telco_churn"

# 1) Create DB if missing
conn = psycopg2.connect(host=HOST, port=PORT, dbname=MASTER_DB, user=USER, password=PASSWORD, sslmode="require")
conn.autocommit = True
cur = conn.cursor()
cur.execute("SELECT 1 FROM pg_database WHERE datname = %s", (DB,))
if cur.fetchone() is None:
    cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(DB)))
cur.close()
conn.close()

# 2) Create table in the created DB
conn = psycopg2.connect(host=HOST, port=PORT, dbname=DB, user=USER, password=PASSWORD, sslmode="require")
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS public.customers (
    customer_id        TEXT PRIMARY KEY,
    gender             TEXT,
    senior_citizen     BOOLEAN,
    partner            BOOLEAN,
    dependents         BOOLEAN,
    tenure             INTEGER,
    phone_service      BOOLEAN,
    multiple_lines     TEXT,
    internet_service   TEXT,
    online_security    TEXT,
    online_backup      TEXT,
    device_protection  TEXT,
    tech_support       TEXT,
    streaming_tv       TEXT,
    streaming_movies   TEXT,
    contract            TEXT,
    paperless_billing   BOOLEAN,
    payment_method      TEXT,
    monthly_charges     NUMERIC(10,2),
    total_charges       NUMERIC(10,2),
    churn               BOOLEAN
);
""")
conn.commit()
cur.close()
conn.close()
print("Done.")

Done.


**Check Table columsn on AWS DB**

In [ ]:
# Hardcoded credentials (as requested)
HOST = "mlprojectdb.cx4k04y48cjc.eu-north-1.rds.amazonaws.com"
PORT = 5432
USER = "postgres"
PASSWORD = "mlpipeline"
MASTER_DB = "postgres"   # connect here to create the database
DB = "telco_churn"
# Connect to the database
conn = psycopg2.connect(
    host=HOST,
    port=PORT,
    dbname=DB,
    user=USER,
    password=PASSWORD,
    sslmode="require"
)

cur = conn.cursor()

table_name = 'customers'
print(f"Schema for table '{table_name}':")
cur.execute("""
    SELECT column_name, data_type, is_nullable
    FROM information_schema.columns
    WHERE table_schema = 'public'
      AND table_name = %s
    ORDER BY ordinal_position;
""", (table_name,))
columns = cur.fetchall()
for col in columns:
    print(f"  {col[0]} | {col[1]} | nullable: {col[2]}")

cur.close()
conn.close()


Schema for table 'customers':
  customer_id | text | nullable: NO
  gender | text | nullable: YES
  senior_citizen | boolean | nullable: YES
  partner | boolean | nullable: YES
  dependents | boolean | nullable: YES
  tenure | integer | nullable: YES
  phone_service | boolean | nullable: YES
  multiple_lines | text | nullable: YES
  internet_service | text | nullable: YES
  online_security | text | nullable: YES
  online_backup | text | nullable: YES
  device_protection | text | nullable: YES
  tech_support | text | nullable: YES
  streaming_tv | text | nullable: YES
  streaming_movies | text | nullable: YES
  contract | text | nullable: YES
  paperless_billing | boolean | nullable: YES
  payment_method | text | nullable: YES
  monthly_charges | numeric | nullable: YES
  total_charges | numeric | nullable: YES
  churn | boolean | nullable: YES


**Insert CSV file into AWS DB Table**

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection parameters (re-using existing kernel state values)
DB_NAME = 'telco_churn'
DB_USER = 'postgres'
DB_PASSWORD = 'mlpipeline'
RDS_HOST = 'mlprojectdb.cx4k04y48cjc.eu-north-1.rds.amazonaws.com'
RDS_PORT = 5432
TABLE_NAME = 'customers'

# Construct the database connection string
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{RDS_HOST}:{RDS_PORT}/{DB_NAME}"

# Create a SQLAlchemy engine
engine = create_engine(DATABASE_URL)

# Path to your CSV file
csv_file_path = 'ref_data.csv'

try:
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)

    # Rename DataFrame columns to match the PostgreSQL table schema
    df = df.rename(columns={
        'customerID': 'customer_id',
        'SeniorCitizen': 'senior_citizen',
        'Partner': 'partner',
        'Dependents': 'dependents',
        'PhoneService': 'phone_service',
        'PaperlessBilling': 'paperless_billing',
        'MonthlyCharges': 'monthly_charges',
        'TotalCharges': 'total_charges',
        'Churn': 'churn',
        'MultipleLines': 'multiple_lines',
        'InternetService': 'internet_service',
        'OnlineSecurity': 'online_security',
        'OnlineBackup': 'online_backup',
        'DeviceProtection': 'device_protection',
        'TechSupport': 'tech_support',
        'StreamingTV': 'streaming_tv',
        'StreamingMovies': 'streaming_movies',
        'Contract': 'contract',
        'PaymentMethod': 'payment_method'
    })

    # Convert 'Yes'/'No' strings to boolean for relevant columns
    boolean_cols = ['senior_citizen', 'partner', 'dependents', 'phone_service', 'paperless_billing', 'churn']
    for col in boolean_cols:
        # Ensure the column exists before trying to convert
        if col in df.columns:
            df[col] = df[col].map({'Yes': True, 'No': False, 1: True, 0: False, '1':True, '0':False})

    # Convert 'TotalCharges' to numeric, coercing errors to NaN
    # Then fill NaN with 0 or a suitable default if needed for the database
    df['total_charges'] = pd.to_numeric(df['total_charges'], errors='coerce').fillna(0)

    # Upload the DataFrame to the PostgreSQL table
    # if_exists='append' will add rows to the existing table, preserving the schema
    # index=False prevents writing the DataFrame index as a column in the DB
    df.to_sql(TABLE_NAME, engine, if_exists='append', index=False)

    print(f"Data from '{csv_file_path}' successfully uploaded to table '{TABLE_NAME}' in '{DB_NAME}'.")

except FileNotFoundError:
    print(f"Error: The file '{csv_file_path}' was not found. Please ensure it's uploaded to your Colab environment.")
except Exception as e:
    print(f"An error occurred during database upload: {e}")


Data from 'ref_data.csv' successfully uploaded to table 'customers' in 'telco_churn'.


**Some queries**

In [47]:
import psycopg2
import pandas as pd

# RDS connection parameters (reusing values from kernel state)
HOST = "mlprojectdb.cx4k04y48cjc.eu-north-1.rds.amazonaws.com"
PORT = 5432
DB = "telco_churn"
USER = "postgres"
PASSWORD = "mlpipeline"

# Connect to the database
conn = psycopg2.connect(
    host=HOST,
    port=PORT,
    dbname=DB,
    user=USER,
    password=PASSWORD,
    sslmode="require"
)

cur = conn.cursor()

# Query 1: Select a few customers
print("\n--- First 5 customers ---")
cur.execute("SELECT * FROM customers LIMIT 5;")
columns = [desc[0] for desc in cur.description]
rows = cur.fetchall()
df_customers = pd.DataFrame(rows, columns=columns)
display(df_customers)

# Query 2: Count all customers
print("\n--- Total customer count ---")
cur.execute("SELECT COUNT(*) FROM customers;")
count = cur.fetchone()[0]
print(f"Total number of customers: {count}")

# cur.close()
# conn.close()


--- First 5 customers ---


,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,...,payment_method,monthly_charges,total_charges,churn,status,notified,updated_at,first_name,last_name,email
0,8844-TONUD,Male,False,True,True,13,True,No,Fiber optic,No,...,Mailed check,96.65,1162.85,True,None,False,2026-01-03 22:58:29.239357,Mariam,Al-Hussein,mariam.al-hussein@outlook.com
1,7306-YDSOI,Male,False,True,True,69,True,No,DSL,Yes,...,Bank transfer (automatic),59.75,4069.90,False,None,False,2026-01-03 22:58:29.239357,Charles,Al-Fahad,charles.al-fahad@hotmail.com
2,5126-RCXYW,Male,False,True,True,54,False,No phone service,DSL,No,...,Credit card (automatic),46.00,2424.05,False,None,False,2026-01-03 22:58:29.239357,Huda,Al-Taher,huda.al-taher@outlook.com
3,6734-GMPVK,Male,False,False,False,5,True,Yes,Fiber optic,No,...,Electronic check,105.30,550.60,False,None,False,2026-01-03 22:58:29.239357,Barbara,Lee,barbara.lee@hotmail.com
4,8327-LZKAS,Female,True,True,False,43,True,Yes,Fiber optic,No,...,Electronic check,107.55,4533.90,True,None,False,2026-01-03 22:58:29.239357,Abdullah,Al-Hakim,abdullah.al-hakim@gmail.com



--- Total customer count ---
Total number of customers: 7043


**ADD Status, Notified, Update_at Columns**

---



In [27]:
# Create a new cursor for schema changes
schema_cur = conn.cursor()

# Add columns: status, notified, updated_at
alter_table_query = """
ALTER TABLE customers
ADD COLUMN IF NOT EXISTS status VARCHAR(20),
ADD COLUMN IF NOT EXISTS notified BOOLEAN,
ADD COLUMN IF NOT EXISTS updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP;
"""

schema_cur.execute(alter_table_query)
conn.commit()

print("Columns 'status', 'notified', and 'updated_at' added successfully.")

schema_cur.close()


Columns 'status', 'notified', and 'updated_at' added successfully.


**ADD first_name, last_name, email Columns**


In [41]:
# Create a new cursor for schema changes
schema_cur = conn.cursor()
# Add first_name and last_name columns
alter_table_query = """
ALTER TABLE customers
ADD COLUMN IF NOT EXISTS first_name VARCHAR(50),
ADD COLUMN IF NOT EXISTS last_name VARCHAR(50),
ADD COLUMN IF NOT EXISTS email VARCHAR(50);
"""

schema_cur.execute(alter_table_query)
conn.commit()

print("Columns 'first_name', 'last_name' and 'email' added successfully.")

schema_cur.close()

Columns 'first_name', 'last_name' and 'email' added successfully.


**ADD random values for the last three columns**

In [43]:
import random

# Roll back any previous failed transaction
conn.rollback()

# Create a new cursor for updates
update_cur = conn.cursor()

# Example pools of first and last names
first_names = [
    "James", "Mary", "John", "Patricia", "Robert", "Jennifer", "Michael", "Linda",
    "William", "Elizabeth", "David", "Barbara", "Richard", "Susan", "Joseph", "Jessica",
    "Thomas", "Sarah", "Charles", "Karen", "Daniel", "Nancy", "Matthew", "Lisa",
    "Ahmed", "Fatima", "Omar", "Aisha", "Ali", "Hassan", "Huda", "Yusuf", "Khalid", "Zainab",
    "Abdullah", "Mariam", "Ibrahim", "Layla", "Nadia", "Sami", "Rania", "Fahad", "Aminah", "Tariq",
    "Salman", "Noor", "Hussein", "Sofia", "Bilal", "Leila", "Mustafa", "Amira", "Imran", "Samira",
    "Yara", "Kareem", "Rashid", "Aya", "Farah", "Jamal", "Zayd", "Lina", "Mahmoud", "Tahira",
    "Anas", "Salah", "Nour", "Rami", "Ayman", "Dina", "Othman", "Hafsa"
]

last_names = [
    "Smith", "Johnson", "Williams", "Brown", "Jones", "Garcia", "Miller", "Davis",
    "Rodriguez", "Martinez", "Hernandez", "Lopez", "Gonzalez", "Wilson", "Anderson",
    "Thomas", "Taylor", "Moore", "Jackson", "Martin", "Lee", "Perez", "Thompson", "White",
    "Al-Farsi", "Hassan", "Al-Mansoori", "Abdullah", "Rahman", "Al-Hakim", "Karim", "Al-Sayed",
    "Najjar", "Zaki", "Qureshi", "El-Sayed", "Farouq", "Al-Amiri", "Hafez", "Saad", "Nasser", "Yousef",
    "Al-Khatib", "Al-Rashid", "Jabari", "Al-Muhammad", "Darwish", "Al-Tamimi", "Bakr", "Al-Zahra", "Fadel", "Al-Hussein",
    "Suleiman", "Al-Baghdadi", "Al-Sharif", "Malik", "Al-Majid", "Hashim", "Al-Amin", "Rashid", "Tariq", "Al-Najjar",
    "Al-Fahad", "Naji", "Al-Qasim", "Omar", "Al-Karim", "Habib", "Al-Taher", "Al-Fayez", "Al-Mutairi", "Zaman"
]


# Fetch all customer_ids
update_cur.execute("SELECT customer_id FROM customers;")
customer_ids = [row[0] for row in update_cur.fetchall()]

# Shuffle customer_ids to randomize assignment
random.shuffle(customer_ids)

# Prepare updates
updates = []
for cust_id in customer_ids:
    first = random.choice(first_names)
    last = random.choice(last_names)
    updates.append((first, last, cust_id))

# Batch update in PostgreSQL
update_query = """
UPDATE customers
SET first_name = %s,
    last_name = %s
WHERE customer_id = %s;
"""

update_cur.executemany(update_query, updates)
conn.commit()

print("First and last names assigned to all customers.")

update_cur.close()


First and last names assigned to all customers.


In [46]:
import random

# Roll back any previous failed transaction
conn.rollback()

# Create a new cursor for email updates
email_cur = conn.cursor()

# Domains to choose from
domains = ["gmail.com", "hotmail.com", "outlook.com"]

# Fetch first_name, last_name, customer_id
email_cur.execute("SELECT customer_id, first_name, last_name FROM customers;")
rows = email_cur.fetchall()

# Dictionary to track used email usernames to avoid duplicates
used_usernames = {}

updates = []

for cust_id, first, last in rows:
    base_username = f"{first.lower()}.{last.lower()}"

    # If base username already used, append a number
    count = used_usernames.get(base_username, 0)
    if count > 0:
        username = f"{base_username}{count}"
    else:
        username = base_username

    # Update the usage counter
    used_usernames[base_username] = count + 1

    # Pick a random domain
    domain = random.choice(domains)

    email = f"{username}@{domain}"

    updates.append((email, cust_id))

# Batch update emails
update_query = """
UPDATE customers
SET email = %s
WHERE customer_id = %s;
"""

email_cur.executemany(update_query, updates)
conn.commit()

print("Emails generated and assigned successfully to all customers.")

email_cur.close()


Emails generated and assigned successfully to all customers.
